# Passo à Passo das Redes Neurais

A criação e treinamento de uma rede neural tem alguns passos que foram um *pipeline* completo.
Nesta aula, vamos ver cada passo para criar e treinar uma rede neural do zero usando PyTorch.

## Imports e configurações iniciais

Agora que a brincadeira está ficando séria, que tal uma sugestão de como organizar o seu código? Para facilitar o entendimento e manutenção do código, mantenha sempre no início os seguintes elementos:
* imports de pacotes
* configuração de **hiperparâmetros**
* definição do hardware padrão utilizado

Nessa aula vamos trabalhar com dados reais, então **vamos precisar de GPU!** Então não se esqueça de mudar as configurações desse ambiente do colab. <br>
Sugiro rodar esse mesmo código sem GPU em outro momento, só pra sentir o gostinho de como a GPU facilitou o uso de redes neurais.


In [0]:
# Basic imports.
import os, sys, time
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

from torch.backends import cudnn
cudnn.benchmark = True

from torch.utils.data import DataLoader
from torch.utils import data

from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

Nesta célula, além da definição do hardware padrão, estão também definidos os hiperparâmetros do nosso modelo. Mais à frente conversaremos um pouco melhor sobre eles.

In [0]:
# Setting predefined arguments.
args = {
    'num_epochs': 20,      # Number of epochs.
    'num_classes': 10,     # Number of classes.
    'lr': 1e-3,            # Learning rate.
    'weight_decay': 5e-4,  # L2 penalty.
    'num_workers': 3,      # Number of workers on data loader.
    'batch_size': 50,      # Mini-batch size.
}

# Setting device (CPU | CUDA)
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

## Carregamento de Dados

#### **Datasets**

O PyTorch possui dois pacotes que trazem datasets prontos para uso.

* Torchtext: https://torchtext.readthedocs.io/en/latest/datasets.html
* Torchvision: https://pytorch.org/docs/stable/torchvision/datasets.html

Como os nomes indicam, são datasets de textos (text) e imagens (vision), duas aplicações onde redes neurais são muito bem sucedidas.

Para aplicações com textos e outros tipos de séries temporais, o carregamento de dados possui nuances que dificultam o entendimento, portanto vamos concentrar no carregamento de imagens.

Para trabalhar com datasets do pacote torchvision, basta
* Importar o pacote
``` python 
from torchvision import datasets 
```
* Carregar o dataset do seu interesse (ex: MNIST)
``` python 
data = datasets.MNIST(root, train=True, transform=None, target_transform=None, download=False)
```

> Note que o torch faz o carregamento das imagens no formato [Pillow](https://pillow.readthedocs.io/en/stable/). Portanto é necessário convertê-las para um tensor usando um [**Transformer**](https://pytorch.org/docs/stable/torchvision/transforms.html).

* Importar o pacote transforms
``` python 
from torchvision import transforms 
```
* preencher o parâmetro ```transform``` do dataset com a função que converte para tensor.
``` python 
transforms.ToTensor() 
```

Pronto! Quando seu dado for carregado, ele passará pela transformação indicada no parâmetro ```transform```, que nesse caso converte o dado para um tensor.


In [0]:
train_set = datasets.MNIST('./', 
                           train=True, 
                           transform=transforms.ToTensor(),
                           download=True)

test_set = datasets.MNIST('./', 
                           train=False, 
                           transform=transforms.ToTensor(),
                           download=False)

print('Amostras de treino: ' + str(len(train_set)) + '\nAmostras de Teste:' + str(len(test_set)))

Cada dataset possui uma implementação específica internamente no pytorch. Verifique o ```type``` da variável que recebeu os dados e veja que se refere a uma classe específica do dataset.

In [0]:
print(type(train_set))

Por se tratar de um conjunto de dados **supervisionado**, cada elemento do dataset é definido por uma tupla `(dado, rótulo)`. Para dados não supervisionados, cada elemento do dataset comporta apenas o dado.

In [0]:
print(type(train_set[0]))

Podemos então iterar no dataset para observar algumas amostras e seus rótulos.

In [0]:
for i in range(3):
  dado, rotulo = train_set[i]
  
  plt.figure()
  plt.imshow(dado[0])
  plt.title('Rotulo: '+ str(rotulo))

Temos um total de 70 mil amostras, mas elas **ainda não estão carregadas na memória** (isso seria bastante custoso). A vantagem da classe ```Dataset``` do Pytorch é que as amostras só são carregadas quando necessário.

Para entender melhor, vamos experimentar a transformação a seguir
```python
transforms.RandomCrop(12)
```
Essa função realiza um recorte aleatório de ```12 x 12``` (pixels) na imagem. Ao carregar a mesma amostra múltiplas vezes, um novo recorte será feito. 

In [0]:
crop_set = datasets.MNIST('./', 
                           train=False, 
                           transform=transforms.RandomCrop(12),
                           download=False)

# Tuple (dado, rótulo)
for i in range(3):
  dado, rotulo = crop_set[0]
  
  plt.figure()
  plt.imshow(dado)
  plt.title('Rótulo: '+ str(rotulo))

Em resumo, cada vez que indexamos um item do dataset, as seguintes operações são realizadas:
* Amostra lida do arquivo e carregada como uma tupla ```(dado, rótulo)```
* As transformações são aplicadas 


#### **Dataloader**



O [Dataloader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader) gerencia muito bem o carregamento de dados para o treinamento de redes neurais, trazendo as funções: 

* Separação dos dados em batches
* Embaralhando os dados
* Carregando batches em paralelo utilizando threads

O uso de threads no carregamento minimiza períodos ociosos de processamento, visto que a leitura de dados em arquivo é um grande gargalo de tempo.

As três funcionalidades que acabamos de conhecer são controladas pelos parâmetros da chamada do DataLoader.
```python
loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)
```


In [0]:
train_loader = DataLoader(train_set, 
                          batch_size=args['batch_size'], 
                          shuffle=True, 
                          num_workers=args['num_workers'])

test_loader = DataLoader(test_set, 
                          batch_size=args['batch_size'], 
                          shuffle=True, 
                          num_workers=args['num_workers'])

O objeto retornado é um **iterador**, podendo ser utilizado para iterar em loops mas não suportando indexação.

In [0]:
for batch in train_loader:
  
  dado, rotulo = batch
  print(dado.size(), rotulo.size())

  plt.imshow(dado[0][0])
  plt.title('Rotulo: '+ str(rotulo[0]) )
  break

Vale a pena visitar o [tutorial de carregamento de dados do PyTorch](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) que introduz o uso das classes Dataset e Dataloader.

## Definindo a Arquitetura

#### **Classe nn.Module**

O [nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) é a classe base para todos os módulos de redes neurais. 

A forma mais organizada de definir modelos em PyTorch é implementando uma classe. Para redes pequenas, como as que estamos aprendendo até o momento, sua importância pode não se destacar, mas modelos maiores e com funcionalidades mais complexas, são mais fáceis de implementar e realizar manutenções dessa forma.

Para implementar uma subclasse da ```nn.Module``` basta definir a subclasse da seguinte forma:
```python
class MinhaRede(nn.Module):
  # resto do código
```

Funções obrigatórias de subclasses da ```nn.Module```.
* ```__init()__```: definição da arquitetura da rede no estado interno da classe.
* ```forward()```: Fluxo da entrada ao longo da rede e retorno da saída.

**Implemente a seguir** um Multi-Layer Perceptron (MLP) intercalando camadas do tipo [`Linear`](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear) com funções de ativação `ReLU`. 


Ainda falaremos mais sobre funções de ativação, mas para entender a importância das funções de ativação não-lineares após cada camada, visite a [demo de Stanford](https://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html). Ao remover as ativações não-lineares a rede se torna incapaz de aprender soluções não-lineares.<br>
A ativação pode ser definida de duas formas:
* Definida no `__init__()` [como uma camada](https://pytorch.org/docs/stable/nn.html?highlight=relu#torch.nn.ReLU) e aplicada no `forward()`
```python
# No __init__()
self.relu = nn.ReLU()
# No forward()
saida = self.relu(entrada)
```

* Apenas aplicada no `forward()` [como uma function](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.relu):
```python
saida = F.relu(entrada)
```

> **Lembrete**: Multi-Layer Perceptrons trabalham somente com dados unidimensionais (vetores). Sendo a imagem com dimensionalidade ```(1, 28, 28)```, precisamos linearizá-la antes de alimentar a rede (lembra da função `view`?). Isso implica que o a entrada da rede deve ser redimensionada para ```input_size = 28 x 28 x 1 = 784```

> **A dimensionalidade das camadas fica a seu critério.** São fixadas apenas a dimensionalidade da entrada (`784`) e da saída (`args['num_classes'] = 10` previamente definida).

In [0]:
class MinhaRede(nn.Module):
  
  def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
    super(MinhaRede, self).__init__()
    
    # Definir a arquitetura
    self.hidden1 = nn.Linear(input_size, hidden1_size)
    self.hidden2 = nn.Linear(hidden1_size, hidden2_size)
    self.output  = nn.Linear(hidden2_size, output_size)

  def forward(self, x):
    
    x = x.view(x.size(0), -1)
    h1 = F.relu(self.hidden1(x))
    h2 = F.relu(self.hidden2(h1))
    output = F.softmax(self.output(h2))
    
    return output

Instanciando a rede

In [0]:
input_size   = 28 * 28
hidden1_size = 128
hidden2_size = 50
out_size     = args['num_classes'] #classes

net = MinhaRede(input_size, hidden1_size, hidden1_size, out_size).to(args['device']) #cast na GPU 

## Função de Perda e Otimizador

Por se tratar de um problema de classificação usaremos a função de custo [`CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss).

Como ainda não vimos a fundo os otimizadores e suas vantagens, usaremos o já conhecido [`SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) da biblioteca `torch.optim`


In [0]:
# Define Loss
criterion = nn.CrossEntropyLoss().to(args['device'])

# Define Optimizer
optimizer = optim.SGD(params=net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

## Fluxo de Treinamento

**Agora implemente a seguir** um fluxo completo de treinamento. Relembrando o passo a passo:

* Iterar nas épocas (Número de épocas definido em `args['num_epochs']`)
* Iterar nos batches (*loaders* pré definidos que retornam uma tupla `(dado, rótulo)`)
* Cast dos dados no dispositivo de hardware (Dispositivo definido em `args['device']`)
* Forward do batch na rede 
* Cálculo da loss (`criterion` previamente definido)
* Cálculo do gradiente a partir da loss (autograd torch)
* Atualização dos pesos (`optimizer.step()`)

Para acompanhar a convergência do seu modelo (e garantir que tudo foi feito certinho), ao final de cada época podemos imprimir a média e o desvio padrão das perdas de cada iteração.

In [0]:
preds_list = []
labels_list = []

for epoch in range(args['num_epochs']):
  start = time.time()

  epoch_loss = []
  for batch in train_loader:
    
    dado, rotulo = batch

    # Cast na GPU
    dado   = dado.to(args['device'])
    rotulo = rotulo.to(args['device'])

    # Forward 
    pred = net(dado)
    loss = criterion(pred, rotulo)
    epoch_loss.append(loss.cpu().data)

    # Predições
    preds = pred.data.max(dim=1)[1].cpu().numpy()
    preds_list.append(preds)
    labels_list.append(rotulo.cpu().numpy())

    # Backward
    loss.backward()
    optimizer.step()

  epoch_loss = np.asarray(epoch_loss)
  acc = metrics.accuracy_score(np.asarray(labels_list).ravel(),
                                 np.asarray(preds_list).ravel())

  end = time.time()

  print("Epoca %d, Loss: %.4f +\- %.4f, Acc: %.2f, Tempo: %.2f" % (epoch, epoch_loss.mean(), epoch_loss.std(), acc*100, end-start) )
